<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%9E%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

### Очистка и подготовка данных


In [ ]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/titanic.csv"

In [ ]:
import pandas as pd
df = pd.read_csv(url)
df

In [ ]:
df.info()
#показывает в том числе сколько пропущенных значений

In [ ]:
# Удаляем столбцы, в которых нет ценной для нас информации
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Округляем стоимость билета до двух знаков после запятой
df['Fare'] = round(df['Fare'], 2)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# можно настраивать и изменять способ удаления данных, например с помощью параметра thresh=2, который оставит строки с более, чем 2 непустыми значениями
# df.dropna()

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [ ]:
#Способ 1: Заменить на константу

#df['Age'].fillna(25)

In [ ]:
# Способ 2: Среднее арифметическее
df['Age'].fillna(df['Age'].mean())

In [ ]:
# Способ 2: Cредние арифметические в зависимости от класса каюты
age_1 = df[df['Pclass'] == 1]['Age'].mean()
age_1

In [ ]:
age_2 = df[df['Pclass'] == 2]['Age'].mean()
age_2

In [ ]:
age_3 = df[df['Pclass'] == 3]['Age'].mean()
age_3

In [ ]:
def fill_age(row):
    if pd.isnull(row['Age']):
        if row['Pclass'] == 1:
            return age_1
        
        if row['Pclass'] == 2:
            return age_2
        
        if row['Pclass'] == 3:
            return age_3
        
    return row['Age']

In [ ]:
df.apply(fill_age, axis=1)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html

# https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group

In [ ]:
df.groupby('Pclass')['Age'].apply(lambda x:x.fillna(x.mean()))

In [ ]:
(df.apply(fill_age, axis = 1)).equals(df.groupby('Pclass')['Age'].apply(lambda x:x.fillna(x.mean())))

### Создаем новый столбец с информацией о том, был ли пассажир на борту один или с родственниками
Столбец должен содержать значение "alone", если он был на борту один (без супруга/супруги, братьев, сестер, детей и родителей) и значение "not alone", если пассажир путешествовал с кем-то из родственников.

- `SibSp` - Количество братьев и сестер / супругов на борту
- `Parch` - число родителей / детей на борту

In [ ]:
# Способ 1: с помощью именной функции и apply
def alone_check(row):
    if row['SibSp'] or row['Parch'] > 0:
        return 'not_alone'
    return 'alone'
    
df['Alone'] = df.apply(alone_check, axis=1)
df

In [ ]:
# Способ 2: с помощью lambda-функции
df['Alone'] = df.apply(lambda x: 'not_alone' if x['SibSp'] or x['Parch'] > 0 else 'alone', axis=1)
df